In [1030]:
"""
importing useful libraries
pandas: manipulating tables and arrays
numpy: manipulate arrays (usually for faster computation than structures like lists)
keyring & teradatasql: saving user credentials and connecting to teradata
pandasql: run sql queries with pandas dataframes
time: calculate run time
datetime: converting strings to datetime objects and vice versa
os: interact with the directories (folder) on this computer
jupyternotify: notifies when cell execution has occurred
"""

import pandas as pd
#import modin.pandas as pd
from pandas import read_sql, ExcelWriter, DataFrame, Series, to_datetime, concat
import numpy as np
from numpy import zeros, where, mean, vectorize, round, ceil
import keyring
import teradatasql
import pandasql
from pandasql import sqldf
import time
import datetime
import os
import saspy
import math
import warnings
import getpass

In [1032]:
promo_ad_year = 2024 #integer
promo_ad_week = 52 #integer
current_week = 42
quarter_num = 4

In [1034]:
# USERNAME: 
username = 'carshen'

In [1036]:
# PASSWORD: 
try:
    password = getpass.getpass()
except:
    print("fail: ", error)
else:
    print("TERADATA PASSWORD")

 ········


TERADATA PASSWORD


In [1038]:
#to set up username and password so you dont need to type it
#only needs to be run once ever


host = 'TDPROD1.ngco.com'

# password=keyring.get_password("TD", username)

queries_folder = os.path.abspath(os.getcwd()) + '\\SQL Queries'

# output_folder = os.path.abspath(os.getcwd()) + '\\Output'

con = teradatasql.connect(None, host=host, user=username, password=password, logmech = 'LDAP')

In [1039]:
profit_panel_query = f'''
sel distinct
T2.RGN_NUM,
T2.RGN_NM_E AS REGION,
T2.BAN_NUM,
T2.BAN_SHRT_DESC_E AS BANNER,
T2.SITE_NUM,
T1.PLANO_ST_DESC AS PLANOGRAM_STATUS, 
T1.PLANO_ENG_NM AS PLANOGRAM_NAME,
T1.PLANO_FR_NM AS PLANOGRAM_NAME_FRENCH
 
 
from RLDMPROD_V.PLANO_STR_SITE_ARTCL T1
 
INNER JOIN RLDMPROD_V.SITE_HIER T2
ON T1.STR_SITE_NUM = T2.SITE_NUM
AND T2.CUST_GRP_CD = 'MK'
 
WHERE T1.PLANO_HIER_5_DESC IN ('GROCERY PP','NATURAL FOODS PP') 
and T1.PLANO_ENG_NM not like '%HABA%'
AND T1.PLANO_ENG_NM not like '%NOFRILLS%'
and T1.PLANO_ST_DESC='Live'
and ban_num not in  ('06','08','25')
'''

In [1042]:
profit_panel_live_db = pd.read_sql(profit_panel_query, con)

C:\Users\carshen\AppData\Local\Temp\ipykernel_16004\1157958213.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  profit_panel_live_db = pd.read_sql(profit_panel_query, con)


In [1044]:
profit_panel_live_db

,RGN_NUM,REGION,BAN_NUM,BANNER,SITE_NUM,PLANOGRAM_STATUS,PLANOGRAM_NAME,PLANOGRAM_NAME_FRENCH
0,0005,Sales Org Atlantic,05,Dominion,0935,Live,GROCERY_PP_02X56_MARKET_U_DAVIDS_TEA,GROCERIE_PP_02X56_MARCHE_U_DAVIDS_TEA
1,0003,Sales Org Ontario,09,Loblaw,1188,Live,GROCERY_PP_02X56_MARKET_P_SNYDERS_PREZTELS,GROCERIE_PP_02X56_MARCHE_P_SNYDERS_PREZTELS
2,0005,Sales Org Atlantic,04,Atlantic Superstore,0368,Live,GROCERY_PP_02X56_MARKET_H_CARNATION_HOT_CHOCOLATE,GROCERIE_PP_02X56_MARCHE_H_CARNATION_HOT_CHOCO...
3,0005,Sales Org Atlantic,05,Dominion,0923,Live,NATURAL_FOODS_PP_01X52_MARKET_G_GREEN_BEAVER,ALIMENTS_NATUREL_PP_01X52_MARCHE_G_PCO_SNACKS
4,0005,Sales Org Atlantic,04,Atlantic Superstore,0317,Live,GROCERY_PP_02X56_MARKET_J_MAYNARDS,GROCERIE_PP_02X56_MARCHE_J_MAYNARDS
...,...,...,...,...,...,...,...,...
4183,0003,Sales Org Ontario,14,Zehrs,0520,Live,NATURAL_FOODS_PP_01X52_MARKET_F_NATURE_CLEAN,ALIMENTS_NATUREL_PP_01X52_MARCHE_F_NATURE_CLEAN
4184,0003,Sales Org Ontario,09,Loblaw,1114,Live,NATURAL_FOODS_PP_01X52_MARKET_O_ENJOY_LIFE,ALIMENTS_NATUREL_PP_01X52_MARCHE_O_ENJOY_LIFE
4185,0003,Sales Org Ontario,14,Zehrs,0520,Live,NATURAL_FOODS_PP_01X52_MARKET_E_GOOD_TO_GO,ALIMENTS_NATUREL_PP_01X52_MARCHE_E_CARBONAUT
4186,0003,Sales Org Ontario,14,Zehrs,0520,Live,GROCERY_PP_02X56_MARKET_N_TEMPTATIONS,GROCERIE_PP_02X56_MARCHE_N_TEMPTATIONS


In [1046]:
def fixed_ID_format(fixture):
    if fixture.startswith("GROCERY_PP_02X56"):
        return 67
    elif fixture.startswith("NATURAL_FOODS_PP_01X52"):
        return 68
    else:
        print("error: unrecognized planogram name - {fixture}")
        return None
def fixed_ID_populate():
    fixtures = profit_panel_live_db.loc[:,'PLANOGRAM_NAME'].tolist()
    fixture_ids = []

    for fixture in fixtures:
        fixture_id = fixed_ID_format(fixture)
        fixture_ids.append(fixture_id)
    return fixture_ids

def fixture_desc_posn_1_populate():
    planogram_names = profit_panel_live_db.loc[:,'PLANOGRAM_NAME'].tolist()
    fix_desc_names = []
    for name in planogram_names:
        if name.startswith("GROCERY_PP_02X56_MARKET_"):
            letter = name[24]
            item_desc = name[26:]
            desc = f"{letter} - GROCERY PP - {item_desc}"
            fix_desc_names.append(desc)
        elif name.startswith('NATURAL_FOODS_PP_01X52_'):
            letter = name[30]
            item_desc = name[32:]
            desc = f"{letter} - NATURAL FOODS PP - {item_desc}" 
            fix_desc_names.append(desc)
        elif name.startswith('GROCERY_PP_02X56_PROV_'):
            letter = name[22]
            item_desc = name[24:]
            desc = f"{letter} - GROCERY PP - {item_desc}"
            fix_desc_names.append(desc)
        else:
            return None
    return fix_desc_names
    
def fixture_desc_format(fixture):
    if fixture == 67:
        return "GROCERY | ENDCAP | PROFIT PANEL"
    elif fixture == 68:
        return "NATURAL FOODS | ENDCAP | PROFIT PANEL"
    else:
        print("error: unrecognized fixture ID - {fixture}")
        return None
    
def fixture_desc_populate():
    fixtures = profit_panels_df.loc[:,'FIXTURE ID'].tolist()
    fixture_desc_list = []

    for fixture in fixtures:
        fixture_desc = fixture_desc_format(fixture)
        fixture_desc_list.append(fixture_desc)
    return fixture_desc_list

def sort_dataframe(profit_panels_df, sort_columns=['SITE ID', 'FIXTURE ID', 'FIXED DESC POSN 1'], ascending=[True, True, True]):
    return profit_panels_df.sort_values(by=sort_columns, ascending=ascending)
    
def calculate_fixt_num(data):
    fixt_num_counters = {}
    fixt_num_list = []
    if isinstance(data, pd.DataFrame):
        for index, row in data.iterrows():
            key = (row['SITE ID'], row['FIXTURE ID'])
            #print(key) 
            if key not in fixt_num_counters:
                fixt_num_counters[key] = 1 
                fixt_num_list.append(fixt_num_counters[key])  # Append here
            elif key in fixt_num_counters:
                fixt_num_counters[key] += 1 
                fixt_num_list.append(fixt_num_counters[key])  # Append here as well
    return fixt_num_list
    
    
def fixture_size_format(fixture_id):
    if fixture_id == 67:
        return (6, 24, 56, "INCHES") 
    elif fixture_id == 68:
        return (6, 14, 52, "INCHES")
    else:
        # Handle other fixture_id values (if needed)
        return None  # Or raise an error

def apply_fixture_dimensions(row):
    dimensions = fixture_size_format(row['FIXTURE ID'])
    if dimensions is not None:
        return dimensions
    else:
        return (None, None, None, None) # Return None for all dimensions if not found
 

In [1048]:
profit_panels_df = pd.DataFrame()

In [1050]:
profit_panels_df['SALES ORG ID'] = profit_panel_live_db['RGN_NUM'].str.strip().astype('int64')
profit_panels_df['SALES ORG DESC'] = profit_panel_live_db['REGION'].str.strip()
profit_panels_df['DIST CHANNEL ID'] = profit_panel_live_db['BAN_NUM'].str.strip().astype('int64')
profit_panels_df['DIST CHANNEL DESC'] = profit_panel_live_db['BANNER'].str.strip()
profit_panels_df['SITE ID'] = profit_panel_live_db['SITE_NUM'].str.strip().astype('int64')
profit_panels_df['store_size'] = 'n/a'
profit_panels_df['FIXTURE ID'] = fixed_ID_populate()
profit_panels_df['FIXTURE DESC'] = fixture_desc_populate()
profit_panels_df['FIXT NUM'] = 0
profit_panels_df[['LENGTH', 'WIDTH', 'HEIGHT', 'UOM']] = profit_panels_df.apply(apply_fixture_dimensions, axis=1, result_type="expand")
profit_panels_df['TOTAL POSN'] = 3
profit_panels_df['FIXED POSN'] = 3
profit_panels_df['FLEX POSN'] = 0
profit_panels_df['MERCH POSN'] = 0
profit_panels_df['FIXED DESC POSN 1'] = fixture_desc_posn_1_populate()
profit_panels_df['FIXED DESC POSN 2'] = "LIVE Q" +str(quarter_num) 
profit_panels_df['FIXED DESC POSN 3'] = profit_panel_live_db['PLANOGRAM_NAME'].str.strip()
#profit_panels_df['FIXT NUM'] = calculate_fixt_num(profit_panels_df)


In [1052]:
profit_panels_df = sort_dataframe(profit_panels_df.copy())
profit_panels_df['FIXT NUM'] = calculate_fixt_num(profit_panels_df)
profit_panels_df

,SALES ORG ID,SALES ORG DESC,DIST CHANNEL ID,DIST CHANNEL DESC,SITE ID,store_size,FIXTURE ID,FIXTURE DESC,FIXT NUM,LENGTH,WIDTH,HEIGHT,UOM,TOTAL POSN,FIXED POSN,FLEX POSN,MERCH POSN,FIXED DESC POSN 1,FIXED DESC POSN 2,FIXED DESC POSN 3
2479,5,Sales Org Atlantic,4,Atlantic Superstore,309,n/a,67,GROCERY | ENDCAP | PROFIT PANEL,1,6,24,56,INCHES,3,3,0,0,A - GROCERY PP - FEASTABLES_CHOCOLATE,LIVE Q4,GROCERY_PP_02X56_MARKET_A_FEASTABLES_CHOCOLATE
1379,5,Sales Org Atlantic,4,Atlantic Superstore,309,n/a,67,GROCERY | ENDCAP | PROFIT PANEL,2,6,24,56,INCHES,3,3,0,0,B - GROCERY PP - NESTLE_CHOCOLATE,LIVE Q4,GROCERY_PP_02X56_MARKET_B_NESTLE_CHOCOLATE
40,5,Sales Org Atlantic,4,Atlantic Superstore,309,n/a,67,GROCERY | ENDCAP | PROFIT PANEL,3,6,24,56,INCHES,3,3,0,0,C - GROCERY PP - ASSORT_CHOC_BARS,LIVE Q4,GROCERY_PP_02X56_MARKET_C_ASSORT_CHOC_BARS
245,5,Sales Org Atlantic,4,Atlantic Superstore,309,n/a,67,GROCERY | ENDCAP | PROFIT PANEL,4,6,24,56,INCHES,3,3,0,0,D - GROCERY PP - DAIRY_MILK_BARS,LIVE Q4,GROCERY_PP_02X56_MARKET_D_DAIRY_MILK_BARS
2338,5,Sales Org Atlantic,4,Atlantic Superstore,309,n/a,67,GROCERY | ENDCAP | PROFIT PANEL,5,6,24,56,INCHES,3,3,0,0,E - GROCERY PP - LINDT_CHOC,LIVE Q4,GROCERY_PP_02X56_MARKET_E_LINDT_CHOC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526,3,Sales Org Ontario,13,Your Ind Grocer,9693,n/a,67,GROCERY | ENDCAP | PROFIT PANEL,10,6,24,56,INCHES,3,3,0,0,J - GROCERY PP - MAYNARDS,LIVE Q4,GROCERY_PP_02X56_MARKET_J_MAYNARDS
219,3,Sales Org Ontario,13,Your Ind Grocer,9693,n/a,67,GROCERY | ENDCAP | PROFIT PANEL,11,6,24,56,INCHES,3,3,0,0,K - GROCERY PP - EXCEL_JUICY_FRUIT,LIVE Q4,GROCERY_PP_02X56_MARKET_K_EXCEL_JUICY_FRUIT
1180,3,Sales Org Ontario,13,Your Ind Grocer,9693,n/a,67,GROCERY | ENDCAP | PROFIT PANEL,12,6,24,56,INCHES,3,3,0,0,L - GROCERY PP - STARBUCKS,LIVE Q4,GROCERY_PP_02X56_MARKET_L_STARBUCKS
2169,3,Sales Org Ontario,13,Your Ind Grocer,9693,n/a,67,GROCERY | ENDCAP | PROFIT PANEL,13,6,24,56,INCHES,3,3,0,0,M - GROCERY PP - BITS_BITES_CRISPERS,LIVE Q4,GROCERY_PP_02X56_MARKET_M_BITS_BITES_CRISPERS


In [998]:
with ExcelWriter('PROFIT PANEL Q' +str(quarter_num) + '.xlsx') as writer:
                 profit_panels_df.to_excel(writer)